Import des librairies
- pixiedust (Kernel Python/Scala)
- time

In [1]:
#import pixiedust
import time
import os
import tweepy
import json
import json_lines


Configuration du nom du fichier utilisé

In [5]:
%env TIME={time.time()}

%env OUTPUT_TWEET_TXT_FILENAME=output_tweet_{os.environ['TIME']}.txt
%env TWEET_JSON_FILENAME=input_tweet_{os.environ['TIME']}.json


env: TIME=1521125436.670453
env: OUTPUT_TWEET_TXT_FILENAME=output_tweet_1521125436.670453.txt
env: TWEET_JSON_FILENAME=input_tweet_1521125436.670453.json


In [6]:
with_geo=False
API_key = json.load(open("API_key.txt", "r"))

auth = tweepy.OAuthHandler(API_key["consumer_key"], API_key["consumer_secret"])
auth.set_access_token(API_key["access_token"], API_key["access_token_secret"])
api = tweepy.API(auth)

keyword = "JO"

tweets = api.search( q=keyword,tweet_mode='extended',include_rts=False,count=3200)

json_tweets=[]
i=0
j=0
while i<1:


    i=i+1
    for tweet in tweets:
        if with_geo==False or tweet._json["place"]!=None or tweet._json["coordinates"]!=None:
            j=j+1
            if len(json_tweets)!=0:
                b=[x for x in json_tweets if json.loads(x)["id"]==tweet._json["id"]]
                if len(b)==0:
                    json_tweets.append(json.dumps(tweet._json))
            else:
                json_tweets.append(json.dumps(tweet._json))

json_str = "\n\r".join(json_tweets)

filename = "tweets-database/"+os.environ['TWEET_JSON_FILENAME']
with open(filename, "w") as f:
    f.write(json_str)

Mise à jour de l'algorithme sur le Datacenter

In [8]:
%%bash
cd Algo1
sbt "push"

[info] Loading settings from idea.sbt ...
[info] Loading global plugins from /Users/Frego/.sbt/1.0/plugins
[info] Loading settings from plugins.sbt ...
[info] Loading project definition from /Users/Frego/Documents/Centrale/4A/OPTION/08_Projet/SparkSenti/Algo1/project
[info] Loading settings from build.sbt ...
[info] Set current project to SparkSenti (in build file:/Users/Frego/Documents/Centrale/4A/OPTION/08_Projet/SparkSenti/Algo1/)
[info] Exclude scala-library-2.11.8.jar from the package
[info] [SparkSenti] Creating a distributable package in /Users/Frego/Documents/Centrale/4A/OPTION/08_Projet/SparkSenti/Algo1/target/pack
[info] [SparkSenti] Copying libraries to /Users/Frego/Documents/Centrale/4A/OPTION/08_Projet/SparkSenti/Algo1/target/pack/lib
[info] [SparkSenti] project jars:
[info] /Users/Frego/Documents/Centrale/4A/OPTION/08_Projet/SparkSenti/Algo1/target/scala-2.11/sparksenti_2.11-0.1.jar
[info] [SparkSenti] project dependencies:
[info] org.scala-lang:scala-reflect:2.11.8
[info

Envoie des tweets collectés sur le datacenter

In [14]:
%%bash
cp tweets-database/$TWEET_JSON_FILENAME Algo1/$TWEET_JSON_FILENAME
cd Algo1
sbt "put $TWEET_JSON_FILENAME"

[info] Loading settings from idea.sbt ...
[info] Loading global plugins from /Users/Frego/.sbt/1.0/plugins
[info] Loading settings from plugins.sbt ...
[info] Loading project definition from /Users/Frego/Documents/Centrale/4A/OPTION/08_Projet/SparkSenti/Algo1/project
[info] Loading settings from build.sbt ...
[info] Set current project to SparkSenti (in build file:/Users/Frego/Documents/Centrale/4A/OPTION/08_Projet/SparkSenti/Algo1/)
spawn scp input_tweet_1521125436.670453.json fregosi1@156.18.90.100:.
fregosi1@156.18.90.100's password: 
input_tweet_1521125436.670453.json            100%  504KB   5.0MB/s   00:00    
spawn ssh fregosi1@156.18.90.100
fregosi1@156.18.90.100's password: 

Welcome to Bright release         7.3

                                                        Based on CentOS Linux 7
                                                                    ID: #000002

Use the following commands to adjust your environment:

'module avail'            - show available modules

Execution de l'analyse de sentiments des tweets colléctés sur le datacenter

In [15]:
%%bash
cd Algo1
rm -rf $TWEET_JSON_FILENAME
sbt "submit $TWEET_JSON_FILENAME"

[info] Loading settings from idea.sbt ...
[info] Loading global plugins from /Users/Frego/.sbt/1.0/plugins
[info] Loading settings from plugins.sbt ...
[info] Loading project definition from /Users/Frego/Documents/Centrale/4A/OPTION/08_Projet/SparkSenti/Algo1/project
[info] Loading settings from build.sbt ...
[info] Set current project to SparkSenti (in build file:/Users/Frego/Documents/Centrale/4A/OPTION/08_Projet/SparkSenti/Algo1/)
spawn ssh fregosi1@156.18.90.100
fregosi1@156.18.90.100's password: 

Welcome to Bright release         7.3

                                                        Based on CentOS Linux 7
                                                                    ID: #000002

Use the following commands to adjust your environment:

'module avail'            - show available modules
'module add <module>'     - adds a module to your environment for this session
'module initadd <module>' - configure module to be loaded at every login

--------------------------------

Récupération du resultat de l'analyse : fichier text (id_tweet sentiment)

In [11]:
%%bash
cd Algo1
sbt "getOutput 'defaultoutput.txt'"
cp defaultoutput.txt ../tweets-database/$OUTPUT_TWEET_TXT_FILENAME
rm -rf defaultoutput.txt

[info] Loading settings from idea.sbt ...
[info] Loading global plugins from /Users/Frego/.sbt/1.0/plugins
[info] Loading settings from plugins.sbt ...
[info] Loading project definition from /Users/Frego/Documents/Centrale/4A/OPTION/08_Projet/SparkSenti/Algo1/project
[info] Loading settings from build.sbt ...
[info] Set current project to SparkSenti (in build file:/Users/Frego/Documents/Centrale/4A/OPTION/08_Projet/SparkSenti/Algo1/)
spawn scp fregosi1@156.18.90.100:'defaultoutput.txt' .
fregosi1@156.18.90.100's password: 
defaultoutput.txt                             100%    0     0.0KB/s   00:00    

[success] Total time: 0 s, completed 15 mars 2018 15:55:19


Pour test de viz (evite de lancer le workflow complet)

In [2]:
%env OUTPUT_TWEET_TXT_FILENAME=output_test.txt
%env TWEET_JSON_FILENAME=input_test.json

env: OUTPUT_TWEET_TXT_FILENAME=output_test.txt
env: TWEET_JSON_FILENAME=input_test.json


Mapping des données pour Viz 

In [12]:
file_output=open("tweets-database/"+os.environ['OUTPUT_TWEET_TXT_FILENAME'],"r", encoding="utf8");
file_tweet=open("tweets-database/"+os.environ['TWEET_JSON_FILENAME'],"r", encoding="utf8");


def parserByTweet(file_output,file_tweet):
    fileText = file_output.read().replace("Négatif","Negatif")
    fileTweets = fileText.split("\n\n")[1:]
    
    tweetList = []
    
    j=0
    for tweet in fileTweets[:-1]:
        j=j+1
        print(j)
        data=tweet.split("\n")
        for i in range(len(data)-1):
            if data[i]=="Score: Positif" or data[i]=="Score: Negatif" or data[i]=="Score: Neutre":
                result=data[i][7:]
                tweet_id=data[i+1][9:]
        
        for tweet_json in json_lines.reader(file_tweet):
            if int(tweet_id)==int(tweet_json["id"]):
                new={}
                new["result"]=result
                new["tweet"]=tweet_json
                tweetList.append(new)  
                break
    return(tweetList)
    
data_to_viz=parserByTweet(file_output,file_tweet)
print(len(data_to_viz))

0
